In [1]:
try:
    import google.colab
    USE_COLAB = True
except:
    USE_COLAB = False
print("Use colab - ", USE_COLAB)

Use colab -  True


In [2]:
if USE_COLAB:
    !pip install --user deeppavlov

# Assignment 2. Language modeling.

This task is devoted to language modeling. Its goal is to write in PyTorch an RNN-based language model. Since word-based language modeling requires long training and is memory-consuming due to large vocabulary, we start with character-based language modeling. We are going to train the model to generate words as sequence of characters. During training we teach it to predict characters of the words in the training set.



## Task 1. Character-based language modeling: data preparation (15 points).

We train the language models on the materials of **Sigmorphon 2018 Shared Task**. First, download the Russian datasets.

In [3]:
!wget https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-train-high
!wget https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-dev
!wget https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-test

--2020-03-31 09:36:57--  https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-train-high
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533309 (521K) [text/plain]
Saving to: ‘russian-train-high.1’

russian-train-high. 100%[===================>] 520.81K  --.-KB/s    in 0.07s   

2020-03-31 09:36:57 (7.47 MB/s) - ‘russian-train-high.1’ saved [533309/533309]

--2020-03-31 09:36:58--  https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-dev
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


**1.1 (1 points)**
All the files contain tab-separated triples ```<lemma>-<form>-<tags>```, where ```<form>``` may contain spaces (*будете соответствовать*). Write a function that loads a list of all word forms, that do not contain spaces.  

In [0]:
def read_infile(infile):
    forms = []
    with open(infile, 'r') as f:
        for line in f:
            lemma, form, tags = line.split('\t')
            if ' ' in form.strip():
                continue
            forms.append(form.lower())
    return forms

In [5]:
train_words = read_infile("russian-train-high")
dev_words = read_infile("russian-dev")
test_words = read_infile("russian-test")
print(len(train_words), len(dev_words), len(test_words))
print(*train_words[:10])

9213 917 922
валлонскому незаконченным истрёпывав личного серьгам необоснованным тюти заросла идеальна гулкой


**1.2 (2 points)** Write a **Vocabulary** class that allows to transform symbols into their indexes. The class should have the method ```__call__``` that applies this transformation to sequences of symbols and batches of sequences as well. You can also use [SimpleVocabulary](https://github.com/deepmipt/DeepPavlov/blob/c10b079b972493220c82a643d47d718d5358c7f4/deeppavlov/core/data/simple_vocab.py#L31) from DeepPavlov. Fit an instance of this class on the training data.

In [24]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary


vocab = SimpleVocabulary(special_tokens=["BEGIN", "END"], save_path='.')
vocab.fit([list(x) for x in train_words])
print(len(vocab))

2020-03-31 09:54:20.365 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead


36


In [29]:
vocab(['BEGIN', 'END'])

[0, 1]

**1.3 (2 points)** Write a **Dataset** class, which should be inherited from ```torch.utils.data.Dataset```. It should take a list of words and the ```vocab``` as initialization arguments.

In [0]:
import torch
from torch.utils.data import Dataset as TorchDataset

class Dataset(TorchDataset):
    
    """Custom data.Dataset compatible with data.DataLoader."""
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __getitem__(self, index):
        """
        Returns one tensor pair (source and target). The source tensor corresponds to the input word,
        with "BEGIN" and "END" symbols attached. The target tensor should contain the answers
        for the language model that obtain these word as input.        
        """
        source = ["BEGIN", *self.data[index]]
        target = [*self.data[index], "END"]

        source_transformed = self.vocab(source)
        target_transformed = self.vocab(target)

        return torch.tensor(source_transformed), torch.tensor(target_transformed)


    def __len__(self):
        return len(self.data)

In [0]:
train_dataset = Dataset(train_words, vocab)
dev_dataset = Dataset(dev_words, vocab)
test_dataset = Dataset(test_words, vocab)

In [67]:
train_dataset[0]

(tensor([ 0, 10,  3, 11, 11,  2,  6,  8, 13,  2, 12, 15]),
 tensor([10,  3, 11, 11,  2,  6,  8, 13,  2, 12, 15,  1]))

**1.4 (3 points)** Use a standard ```torch.utils.data.DataLoader``` to obtain an iterable over batches. Print the shape of first 10 input batches with ```batch_size=1```.

In [68]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1)
for i, (source, target) in enumerate(train_dataloader):
    if i == 10:
        break

    print(f'Butch number {i}:')
    print(source, target, sep='\n')

Butch number 0:
tensor([[ 0, 10,  3, 11, 11,  2,  6,  8, 13,  2, 12, 15]])
tensor([[10,  3, 11, 11,  2,  6,  8, 13,  2, 12, 15,  1]])
Butch number 1:
tensor([[ 0,  6,  4, 21,  3, 13,  2,  6, 24,  4,  6,  6, 16, 12]])
tensor([[ 6,  4, 21,  3, 13,  2,  6, 24,  4,  6,  6, 16, 12,  1]])
Butch number 2:
tensor([[ 0,  5,  8,  9,  7, 32, 14, 16, 10,  3, 10]])
tensor([[ 5,  8,  9,  7, 32, 14, 16, 10,  3, 10,  1]])
Butch number 3:
tensor([[ 0, 11,  5, 24,  6,  2, 19,  2]])
tensor([[11,  5, 24,  6,  2, 19,  2,  1]])
Butch number 4:
tensor([[ 0,  8,  4,  7, 22, 19,  3, 12]])
tensor([[ 8,  4,  7, 22, 19,  3, 12,  1]])
Butch number 5:
tensor([[ 0,  6,  4,  2, 23,  2,  8,  6,  2, 10,  3,  6,  6, 16, 12]])
tensor([[ 6,  4,  2, 23,  2,  8,  6,  2, 10,  3,  6,  6, 16, 12,  1]])
Butch number 6:
tensor([[ 0,  9, 27,  9,  5]])
tensor([[ 9, 27,  9,  5,  1]])
Butch number 7:
tensor([[ 0, 21,  3,  7,  2,  8, 11,  3]])
tensor([[21,  3,  7,  2,  8, 11,  3,  1]])
Butch number 8:
tensor([[ 0,  5, 17,  4,  3, 11,

**(1.5) 1 point** Explain, why this does not work with larger batch size.

Это происходит из-за того, что слова могут иметь разную длина, а тенсзоры поддерживают только равные длины по всем осям

**(1.6) 5 points** Write a function **collate** that allows you to deal with batches of greater size. See [discussion](https://discuss.pytorch.org/t/dataloader-for-various-length-of-data/6418/8) for an example. Implement your function as a class ```__call__``` method to make it more flexible.

In [0]:
def pad_tensor(vec, length, dim, pad_symbol):
    """
    Pads a vector ``vec`` up to length ``length`` along axis ``dim`` with pad symbol ``pad_symbol``.
    """
    pad_shape = list(vec.shape)
    pad_shape[dim] = length - pad_shape[dim]
    pad_vec = torch.zeros(pad_shape, dtype=vec.dtype) + pad_symbol
    return torch.cat([vec, pad_vec], dim=dim)

class Padder:
    def __init__(self, dim=0, pad_symbol=0):
        self.dim = dim
        self.pad_symbol = pad_symbol
    
    def __call__(self, batch):
        max_len = -1
        for single_batch in batch:
            for row in single_batch:
                max_len = max(max_len, row.shape[self.dim])
        
        batches = [[] for _ in range(len(batch[0]))]
        for single_batch in batch:
            for i, row in enumerate(single_batch):
                batches[i].append(pad_tensor(row, max_len, self.dim, self.pad_symbol))

        for i in range(len(batches)):
            batches[i] = torch.cat(batches[i])
        return batches

In [70]:
pad_tensor(torch.tensor([1, 2, 3]), 5, 0, 0)

tensor([1, 2, 3, 0, 0])

**(1.7) 1 points** Again, use ```torch.utils.data.DataLoader``` to obtain an iterable over batches. Print the shape of first 10 input batches with the batch size you like.

In [71]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, collate_fn=Padder(pad_symbol=vocab('END')))
for i, (source, target) in enumerate(train_dataloader):
    if i == 10:
        break

    print(f'Butch number {i}:')
    print(source, target, sep='\n')

Butch number 0:
tensor([ 0, 10,  3, 11, 11,  2,  6,  8, 13,  2, 12, 15,  1,  1,  0,  6,  4, 21,
         3, 13,  2,  6, 24,  4,  6,  6, 16, 12])
tensor([10,  3, 11, 11,  2,  6,  8, 13,  2, 12, 15,  1,  1,  1,  6,  4, 21,  3,
        13,  2,  6, 24,  4,  6,  6, 16, 12,  1])
Butch number 1:
tensor([ 0,  5,  8,  9,  7, 32, 14, 16, 10,  3, 10,  0, 11,  5, 24,  6,  2, 19,
         2,  1,  1,  1])
tensor([ 5,  8,  9,  7, 32, 14, 16, 10,  3, 10,  1, 11,  5, 24,  6,  2, 19,  2,
         1,  1,  1,  1])
Butch number 2:
tensor([ 0,  8,  4,  7, 22, 19,  3, 12,  1,  1,  1,  1,  1,  1,  1,  0,  6,  4,
         2, 23,  2,  8,  6,  2, 10,  3,  6,  6, 16, 12])
tensor([ 8,  4,  7, 22, 19,  3, 12,  1,  1,  1,  1,  1,  1,  1,  1,  6,  4,  2,
        23,  2,  8,  6,  2, 10,  3,  6,  6, 16, 12,  1])
Butch number 3:
tensor([ 0,  9, 27,  9,  5,  1,  1,  1,  0, 21,  3,  7,  2,  8, 11,  3])
tensor([ 9, 27,  9,  5,  1,  1,  1,  1, 21,  3,  7,  2,  8, 11,  3,  1])
Butch number 4:
tensor([ 0,  5, 17,  4,  3, 11, 

## Task 2. Character-based language modeling. (35 points)

**2.1 (5 points)** Write a network that performs language modeling. It should include three layers:
1. **Embedding** layer that transforms input symbols into vectors.
2. An **RNN** layer that outputs a sequence of hidden states (you may use https://pytorch.org/docs/stable/nn.html#gru).
3. A **Linear** layer with ``softmax`` activation that produces the output distribution for each symbol.

In [0]:
import torch.nn as nn

class RNNLM(nn.Module):
    def __init__(self, vocab_size, embeddings_dim, hidden_size):
        super(RNNLM, self).__init__()
        """
        == YOUR CODE HERE ==
        """
        
    def forward(self, inputs, hidden=None):
        """
        == YOUR CODE HERE ==
        """

**2.2 (1 points)** Write a function ``validate_on_batch`` that takes as input a model, a batch of inputs and a batch of outputs, and the loss criterion, and outputs the loss tensor for the whole batch. This loss should not be normalized.

In [0]:
def validate_on_batch(model, criterion, x, y):
    """
    == YOUR CODE HERE ==
    """

**2.3 (1 points)** Write a function ``train_on_batch`` that accepts all the arguments of ``validate_on_batch`` and also an optimizer, calculates loss and makes a single step of gradient optimization. This function should call ``validate_on_batch`` inside.

In [0]:
def train_on_batch(model, criterion, x, y, optimizer):
    """
    == YOUR CODE HERE ==
    """

**2.4 (3 points)** Write a training loop. You should define your ``RNNLM`` model, the criterion, the optimizer and the hyperparameters (number of epochs and batch size). Then train the model for a required number of epochs. On each epoch evaluate the average training loss and the average loss on the validation set. 

**2.5 (3 points)** Do not forget to average your loss over only non-padding symbols, otherwise it will be too optimistic.

In [0]:
"""
== YOUR CODE HERE ==
"""

**2.6 (5 points)** Write a function **predict_on_batch** that outputs letter probabilities of all words in the batch.

In [0]:
"""
== YOUR CODE HERE ==
"""

**2.7 (1 points)** Calculate the letter probabilities for all words in the test dataset. Print them for 20 last words. Do not forget to disable shuffling in the ``DataLoader``.

In [0]:
"""
== YOUR CODE HERE ==
"""

**2.8 (5 points)** Write a function that generates a single word (sequence of indexes) given the model. Do not forget about the hidden state! Be careful about start and end symbol indexes. Use ``torch.multinomial`` for sampling.

In [0]:
def generate(model, max_length=20, start_index=1, end_index=2):
    """
    == YOUR CODE HERE ==
    """

**2.9 (1 points)** Use ``generate`` to sample 20 pseudowords. Do not forget to transform indexes to letters.

In [0]:
for i in range(20):
    """
    == YOUR CODE HERE ==
    """

**(2.10) 5 points** Write a batched version of the generation function. You should sample the following symbol only for the words that are not finished yet, so apply a boolean mask to trace active words.

In [0]:
def generate_batch(model, batch_size, max_length = 20, start_index=1, end_index=2):
    """
    == YOUR CODE HERE ==
    """

In [0]:
generated = []
for _ in range(2):
    generated += generate_batch(model, batch_size=10)
"""
== YOUR CODE HERE ==
"""
for elem in transformed:
    print("".join(elem))

**(2.11) 5 points** Experiment with the type of RNN, number of layers, units and/or dropout to improve the perplexity of the model.